## NDVI after the Las Conchas fire in New Mexico

This notebook will be used to download NDVI data using the NASA appeeaars api.

In [1]:
# Import Libraries

import pathlib
import os

import earthpy
import hvplot.pandas
import hvplot.xarray
import matplotlib

import geopandas as gpd
import pandas as pd
import rioxarray as rxr
import xarray as xr
import holoviews as hv
import earthpy.api.appeears as eaapp

In [2]:
# Make a project and a directory

project = earthpy.Project('Las Conchas Vegetation', dirname='fire_boundary')

In [3]:
# Read in Fire Boundary shapefile

fire_bound_gdf = gpd.read_file(project.project_dir / 'nm3581210654120110626_20110624_20120618_burn_bndy.shp')
fire_bound_gdf

,Event_ID,irwinID,Incid_Name,Incid_Type,Map_ID,Map_Prog,Asmnt_Type,BurnBndAc,BurnBndLat,BurnBndLon,...,Perim_ID,dNBR_offst,dNBR_stdDv,NoData_T,IncGreen_T,Low_T,Mod_T,High_T,Comment,geometry
0,NM3581210654120110626,None,LAS CONCHAS,Wildfire,3962,MTBS,Extended,150874,35.87,-106.423,...,503403520110928,-5,-9999,-970,-150,5,177,380,None,"POLYGON ((-917592.246 1492208.61, -917565.51 1..."


In [4]:
# Check CRS
fire_bound_gdf.crs # ESRI:102039

# Change CRS to EPSG: 4326
fire_bound_gdf = fire_bound_gdf.to_crs(4326)
print(fire_bound_gdf.crs)

EPSG:4326


In [ ]:
# Quick plot to make sure the boundary looks right

fire_bound_gdf.hvplot(
    geo=True, 
    tiles='EsriNatGeo', 
    frame_width=500,
    legend=False, fill_color='red', edge_color='white'
    # This parameter makes all the column values in the dataset visible.
    #hover_cols='all'
)


This looks like what we want, so we're set to download MODIS NDVI data now!

In [5]:
# Initialize AppeearsDownloader for MODIS NDVI data

# set parameters
ndvi_downloader = eaapp.AppeearsDownloader(

    # give download a name
    download_key = "las-conchas-fire-ndvi",

    # put data in the directory we defined
    project = project,

    # specify MODIS product we want
    product = 'MOD13Q1.061',
    layer = '_250m_16_days_NDVI',

    # choose a start date and end data
    start_date = "05-01",
    end_date = "09-30",

    # recurring means I want those dates over multiple years
    recurring = True,

    # specify the range of years
    year_range = [2006, 2021],

    # specify the polygon I want to get NDVI data for
    polygon = fire_bound_gdf
)

In [6]:
# download the prepared download - this can take a while!
ndvi_downloader.download_files(cache=True)

No stored credentials found for urs.earthdata.nasa.gov. Please log in.


/opt/conda/lib/python3.11/site-packages/earthpy/api/auth.py:192: UserWarning: Setting credentials not supported for 'netrc' backend.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/earthpy/api/auth.py:196: UserWarning: Failed to store credentials with 'keyring': No recommended backend was available. Install a recommended 3rd party backend package; or, install the keyrings.alt package if you want to use the non-recommended backends. See https://pypi.org/project/keyring for details.
  warnings.warn(


Credentials stored using 'env' backend.


In [7]:
# get a sorted list of NDVI file paths
ndvi_paths = sorted(list(project.project_dir.rglob('*NDVI*.tif')))

len(ndvi_paths)

176